## Addestramento di un modello personalizzato di rilevamento oggetti con TensorFlow Lite Model Maker

In questo notebook Jupyter sarà effettuato l'addestramento e la valutazione di un modello personalizzato di rilevamento oggetti in grado di rilevare volti e testi. Il modello TensorFlow ottenuto dall'addestramento sarà convertito in un modello TensorFlow Lite. Quest'ultimo sarà importato nell'applicazione Android per eseguire il rilevamento di volti e scritte.

## Preparazione

### Installare i pacchetti richiesti

Partiamo con l'installazione dei pacchetti richiesti:
* **tflite-model-maker:** pacchetto del Model Maker, preso direttamente dalla sua [repository](https://github.com/tensorflow/examples/tree/master/tensorflow_examples/lite/model_maker) su GitHub. 
* **tflite-support:** la libreria pycocotools che sarà usata per la valutazione dei modelli.

In [ ]:
!pip install -q tflite-model-maker
!pip install -q tflite-support

Importiamo i pacchetti necessari.

In [2]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat, QuantizationConfig
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

### Preparazione del dataset

Questo dataset contiene 25.000 immagini in cui ci sono i due oggetti da rilevare: visi e testi. 

Scarichiamo da Google Drive l'archivio zip del dataset, lo estraiamo e infine eliminiamo l'archivio zip per risparmiare spazio sul disco.

In [ ]:
!gdown '1UVNMX8GBapg30CxnW7kJe0XEZsHUu0F0'
!unzip -q dataset.zip
!rm dataset.zip

## Addestramento del modello di rilevamento oggetti

### Passo 1: Caricare il dataset

* Le immagini in `train_data` sono usate per addestrare il modello di rilevamento oggetti.
* Le immagini in `val_data` sono usate per controllare se il modello riesce a generalizzare bene analizzando immagini mai viste prima.

In [ ]:
train_data = object_detector.DataLoader.from_pascal_voc(
    '/content/dataset/train',
    '/content/dataset/train',
    ['faccia', 'testo']
)

val_data = object_detector.DataLoader.from_pascal_voc(
    '/content/dataset/validation',
    '/content/dataset/validation',
    ['faccia', 'testo']
)

### Passo 2: Selezionare l'architettura del modello

La EfficientDet-Lite[0-4] è una famiglia di modelli di rilevamento oggetti di tipo mobile/IoT-friendly derivati dell'architettura [EfficientDet](https://arxiv.org/abs/1911.09070).

Di seguito, viene riportato un confronto tra le prestazioni di ciascun modello EfficientDet-Lite.

|    Architettura    | Dimensione (MB)* | Latenza (ms)** | Precisione media*** |
|--------------------|------------------|----------------|---------------------|
| EfficientDet-Lite0 | 4.4              | 37             | 25.69%              |
| EfficientDet-Lite1 | 5.8              | 49             | 30.55%              |
| EfficientDet-Lite2 | 7.2              | 69             | 33.97%              |
| EfficientDet-Lite3 | 11.4             | 116            | 37.70%              |
| EfficientDet-Lite4 | 19.9             | 260            | 41.96%              |

<i> * Dimensione dei modelli con quantizzazione full integer. <br/>
** Latenza misurata su un Pixel 4 che usa 4 thread sulla CPU. <br/>
*** Precisione media di tipo mAP (mean Average Precision) misurata con il validation set del dataset COCO 2017.
</i>

Nel nostro progetto abbiamo usato la EfficientDet-Lite0 per la sua velocità e la EfficientDet-Lite2 per una maggiore precisione.

In [ ]:
spec = object_detector.EfficientDetSpec(model_name='efficientdet-lite0', uri='https://tfhub.dev/tensorflow/efficientdet/lite0/feature-vector/1', hparams={'max_instances_per_image': 10000})

#spec = object_detector.EfficientDetSpec(model_name='efficientdet-lite2', uri='https://tfhub.dev/tensorflow/efficientdet/lite2/feature-vector/1', hparams={'max_instances_per_image': 10000})

### Passo 3: Addestrare il modello TensorFlow con il training set

* `epochs = 30`: durante l'addestramento il training set sarà esaminato per 30 volte. E' importante stare attenti all'accuratezza della validation e fermarsi quando il valore di `val_loss` smette di diminuire per evitare overfitting.
* `batch_size = 20`: saranno necessari 1.000 step per esaminare tutte le 20.000 immagini del training set.
* `train_whole_model=True`: usato per il fine-tuning dell'intero modello, evitando di allenare solamente il layer head per migliorare l'accuratezza. Purtroppo, questa opzione dilata i tempi di addestramento.

L'addestramento con Model Maker è basato sul transfer learning, ovvero sulla riconversione della rete preaddestrata (selezionata in `spec`) per riconoscere volti e testi. 

In [ ]:
model = object_detector.create(train_data, model_spec=spec, batch_size=20, train_whole_model=True, epochs=30, validation_data=val_data)

### Passo 4: Valutare il modello TensorFlow

Dopo aver addestrato il modello di rilevamento oggetti utilizzando le immagini del training set, si utilizzano le immagini del validation set per valutare le sue prestazioni.

In questo caso, la dimensione di default del batch è 64.

Le metriche di valutazione sono le stesse usate da [COCO](https://cocodataset.org/#detection-eval).

In [ ]:
model.evaluate(val_data)

### Passo 5: Esportare come modello TensorFlow Lite

Esportare il modello di rilevamento oggetti addestrato nel formato TensorFlow Lite. La tecnica di quantizzazione post-allenamento predefinita è la [quantizzazione full integer](https://www.tensorflow.org/lite/performance/post_training_integer_quant). Ciò consente al modello TensorFlow Lite di essere più piccolo e veloce quando sarà integrato nella nostra applicazione Android.

In [ ]:
model.export(export_dir='.', tflite_filename='android.tflite')

### Passo 6:  Valutare il modello TensorFlow Lite

Diversi fattori possono influenzare la precisione del modello durante l'esportazione in TensorFlow Lite:
* Usiamo la [quantizzazione fill-integer](https://www.tensorflow.org/lite/performance/model_optimization) che converte i pesi e le operazioni della rete neurale da Float32 a Int8; riducendo la dimensione del modello fino a 4 volte, velocizzandolo di circa 3 volte ma perdendo anche precisione.
* Nel post-processing, il modello TensorFlow utilizza per ogni classe il [NMS (Non-Max Supression)](https://www.coursera.org/lecture/convolutional-neural-networks/non-max-suppression-dvrjH); mentre il modello TensorFlow Lite usa il NMS globalmente con il conseguente aumento di velocità a discapito dell'accuratezza.
* Keras è in grado di rilevare massimo 100 oggetti, mentre TensorFlow Lite solamente 25.

In [ ]:
model.evaluate_tflite('android.tflite', val_data)

### Passo 7:  Scaricare il modello TensorFlow Lite

Scaricare il modello TensorFlow Lite ottenuto e importarlo nell'applicazione Android.

In [ ]:
from google.colab import files
files.download('android.tflite')